# BlogFeedback Clf

## 1. Processing

In [44]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split

import os 

train_df = pd.read_csv("merged_df_pruned_with_target.csv", index_col = 0 )

name_list = train_df.columns

/var/folders/gd/lsdt5y754fl3r9jg4qztyy180000gn/T/ipykernel_43298/1712114100.py:18: DtypeWarning: Columns (284) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("merged_df_pruned_with_target.csv", index_col = 0 )


In [23]:
train_df['new_target'] = train_df.num_comments_next_24_TB >= 10
train_df

,mean_51,sd_51,min_51,max_51,median_51,mean_52,sd_52,min_52,max_52,median_52,...,Sun_TP,num_parent,min_num_comments_P,max_num_comments_P,mean_num_comments_P,num_comments_next_24_TB,dow_P,high_engagement_v3,length_bucket,new_target
0,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.441880,0.0,377.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,Thu_P,0,0,False
1,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.441880,0.0,377.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Wed_P,0,0,False
4,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.441880,0.0,377.0,3.0,...,0.0,0.0,0.0,0.0,0.0,27.0,Thu_P,0,0,True
5,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.441880,0.0,377.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Wed_P,0,0,False
8,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.441880,0.0,377.0,3.0,...,0.0,0.0,0.0,0.0,0.0,9.0,Thu_P,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52392,33.00000,0.000000,33.0,33.0,33.0,11.00000,15.556349,0.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Mon_P,0,3k-4k,False
52393,33.00000,0.000000,33.0,33.0,33.0,11.00000,15.556349,0.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Mon_P,0,3k-4k,False
52394,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Thu_P,0,0,False
52395,0.00000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Thu_P,0,0,False


In [24]:
# Remove dependent cols (col 1...50)
train_df_cleaned = train_df.iloc[:,50:]

# Drop the dow_P as 
train_df_cleaned = train_df_cleaned.drop(columns = ['dow_P', 'length_bucket', 'min_num_comments_P', 'max_num_comments_P', 'mean_num_comments_P', 'high_engagement_v3', 'num_comments_next_24_TB'])

Y = train_df_cleaned.new_target
X = train_df_cleaned.drop(columns = ['new_target'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

def resample_data(df_data, factor):
    # Identify the larger and smaller class
    is_larger_class = (df_data['new_target'] == False)
    is_smaller_class = (df_data['new_target'] == True)

    # Get the indices of each class
    larger_class_indices = np.where(is_larger_class)[0]
    smaller_class_indices = np.where(is_smaller_class)[0]

    # Calculate the target number of samples for both classes
    target_smaller_class_samples = len(smaller_class_indices) * factor

    # Oversample the smaller class using bootstrapping
    oversampled_smaller_class_indices = np.random.choice(
        smaller_class_indices, size=target_smaller_class_samples, replace=True
    )

    # Calculate the number of additional rows added for the larger class
    num_additional_rows =  len(oversampled_smaller_class_indices) - len(smaller_class_indices)

    # Undersample the larger class
    num_larger_class_samples = len(larger_class_indices)
    undersampled_larger_class_indices = np.random.choice(
        larger_class_indices, size=num_larger_class_samples - num_additional_rows, replace=False
    )

    # Combine the oversampled smaller class and undersampled larger class
    combined_indices = np.concatenate([oversampled_smaller_class_indices, undersampled_larger_class_indices])

    # Create the resampled DataFrame
    df_data_resampled = df_data.iloc[combined_indices]

    return df_data_resampled

df_data = pd.concat([X_train, Y_train], axis = 1)
df_data_sampled = resample_data(df_data, 6)
Y_train2 = df_data_sampled.new_target
X_train2 = df_data_sampled.drop(columns = ['new_target'])

In [25]:
Y_train.value_counts()

new_target
False    29703
True      2804
Name: count, dtype: int64

In [26]:
Y_train2.value_counts()

new_target
True     16824
False    15683
Name: count, dtype: int64

In [80]:
# Define the path to the folder you want to change to
folder_name = "bf_data"  # Replace with the name of your target folder

# Create the full path by joining the current directory and the folder name
folder_path = os.path.join("/Users/lexy/Desktop/BlogFeedback", folder_name)

# Change the current working directory to the target folder
os.chdir(folder_path)


# Reading in Test Data
def load_test_data(name_list_final):
    test_filelist = []
    for filename in os.listdir('.'):
        if 'test' in filename:
            #if((filename.split('.')[1] == '02') | (filename.split('.')[1] == '03')):
            test_filelist.append(filename)
    df_test = pd.DataFrame(columns = name_list_final)

    for testfile in test_filelist:
        df_temp = pd.read_csv(testfile, header = None, names = name_list_final)
        df_test = pd.concat([df_test, df_temp], ignore_index = True)

    stat_col_list = []
    for i in range(51,61,1):
        stat_col_list.append('mean' + '_' + str(i))
        stat_col_list.append('sd' + '_' + str(i))
        stat_col_list.append('min' + '_' + str(i))
        stat_col_list.append('max' + '_' + str(i))
        stat_col_list.append('median' + '_' + str(i))

    df_test_stats = df_test[stat_col_list]
    df_test['target'] = np.where((df_test.num_comments_next_24_TB) >= 10, 1, 0)

    df_test.drop(stat_col_list, axis = 1, inplace = True)

    return df_test

name_list = train_df.columns
name_list = name_list.drop(['dow_P', 'high_engagement_v3', 'length_bucket', 'min_num_comments_P', 'max_num_comments_P', 'mean_num_comments_P'])
actual_test_data = load_test_data(name_list)

Y_actual_test_data = actual_test_data.rename(columns = {'target':'new_target'}).new_target
X_actual_test_data = actual_test_data.drop(['target', 'num_comments_next_24_TB'], axis = 1)

In [30]:
print("Number of missing values:", actual_test_data.isnull().values.sum())

Number of missing values: 0


## 2. Modelling

### 2.1. Gradient Boost

In [46]:
clf = GradientBoostingClassifier(
                         learning_rate=0.01, # contribution of each tree
                         n_estimators=500, # large values -> better performance
                         random_state=127
                     ) 
clf.fit(X_train2, Y_train2)
test_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [47]:
print(cm:=classification_report(Y_test, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_test, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_test, test_proba[:,1]))

              precision    recall  f1-score   support

       False   0.989454  0.850008  0.914446     12694
        True   0.370994  0.907108  0.526612      1238

    accuracy                       0.855082     13932
   macro avg   0.680224  0.878558  0.720529     13932
weighted avg   0.934498  0.855082  0.879983     13932

TN: 10790
FP: 1904
FN: 115
TP: 1123
Recall: 0.907108239095315
Precision: 0.3709943838784275
AUC: 0.9467303953147953


In [52]:
clf = GradientBoostingClassifier(
                         # loss='log_loss', # loss function to optimize ???
                         learning_rate=0.1, # contribution of each tree
                         n_estimators=100, # large values -> better performance
                         subsample=1, #  values < 1.0 -> low variance and high bias
                         criterion='friedman_mse', # measure the quality of a split
                         max_depth=3, # tree depth limits the number of nodes
                         random_state=127
                     ) 
clf.fit(X_train2, Y_train2)
test_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [53]:
print(cm:=classification_report(Y_test, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_test, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_test, test_proba[:,1]))

              precision    recall  f1-score   support

       False   0.986090  0.871199  0.925091     12694
        True   0.398233  0.873990  0.547155      1238

    accuracy                       0.871447     13932
   macro avg   0.692162  0.872595  0.736123     13932
weighted avg   0.933853  0.871447  0.891508     13932

TN: 11059
FP: 1635
FN: 156
TP: 1082
Recall: 0.8739903069466882
Precision: 0.39823334560176665
AUC: 0.9427456473273089


### 2.2. Random Forest

In [50]:
clf = RandomForestClassifier(
                         n_estimators=100, 
                         criterion='gini', 
                         max_depth=3, 
                         n_jobs=-1, 
                         max_samples=0.3, 
                         random_state=127
                     )

clf.fit(X_train2, Y_train2)
test_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

print(cm:=classification_report(Y_test, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_test, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_test, test_proba[:,1]))

              precision    recall  f1-score   support

       False   0.982148  0.836458  0.903467     12694
        True   0.334829  0.844103  0.479468      1238

    accuracy                       0.837138     13932
   macro avg   0.658488  0.840281  0.691468     13932
weighted avg   0.924627  0.837138  0.865791     13932

TN: 10618
FP: 2076
FN: 193
TP: 1045
Recall: 0.8441033925686591
Precision: 0.3348285805831464
AUC: 0.9066006404511513


## 3. Final Model (Hyperparameter Tuning)

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

def tune_gradient_boosting_hyperparameters(X, y):
    # Define the hyperparameter grid to search
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 3, 4],
        'min_samples_leaf': [1, 2, 3],
    }

    # Create a Gradient Boosting classifier
    clf = GradientBoostingClassifier()

    # Define AUC as the scoring metric
    auc_scorer = make_scorer(roc_auc_score)

    # Initialize GridSearchCV with cross-validation and AUC scoring
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=auc_scorer, n_jobs=-1)

    # Fit the model to find the best hyperparameters
    grid_search.fit(X, y)

    # Get the best hyperparameters and their corresponding AUC score
    best_params = grid_search.best_params_
    best_auc_score = grid_search.best_score_

    return best_params, best_auc_score

# Example usage:
# Replace 'X' and 'y' with your feature matrix and target variable
best_params, best_auc_score = tune_gradient_boosting_hyperparameters(X_train2, Y_train2)
print("Best Hyperparameters:", best_params)
print("Best AUC Score:", best_auc_score)

KeyboardInterrupt: 

In [57]:
clf = GradientBoostingClassifier(
                         learning_rate=0.01, # contribution of each tree
                         n_estimators=500, # large values -> better performance
                         random_state=127
                     ) 
clf.fit(X_train2, Y_train2)

              precision    recall  f1-score   support

           0   0.986486  0.701923  0.820225       104
           1   0.243902  0.909091  0.384615        11

    accuracy                       0.721739       115
   macro avg   0.615194  0.805507  0.602420       115
weighted avg   0.915457  0.721739  0.778558       115

TN: 73
FP: 31
FN: 1
TP: 10
Recall: 0.9090909090909091
Precision: 0.24390243902439024
AUC: 0.8011363636363635


In [83]:
test_proba = clf.predict_proba(X_actual_test_data)
y_pred = clf.predict(X_actual_test_data)

print(cm:=classification_report(Y_actual_test_data, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_actual_test_data, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_actual_test_data, test_proba[:,1]))

              precision    recall  f1-score   support

           0   0.981636  0.749151  0.849779      7064
           1   0.206449  0.823214  0.330111       560

    accuracy                       0.754591      7624
   macro avg   0.594042  0.786182  0.589945      7624
weighted avg   0.924697  0.754591  0.811608      7624

TN: 5292
FP: 1772
FN: 99
TP: 461
Recall: 0.8232142857142857
Precision: 0.206448723690103
AUC: 0.860502826201262


In [87]:
clf = GradientBoostingClassifier(
                         # loss='log_loss', # loss function to optimize ???
                         learning_rate=0.1, # contribution of each tree
                         n_estimators=100, # large values -> better performance
                         subsample=1, #  values < 1.0 -> low variance and high bias
                         criterion='friedman_mse', # measure the quality of a split
                         max_depth=3, # tree depth limits the number of nodes
                         random_state=127
                     ) 
clf.fit(X_train2, Y_train2)

GradientBoostingClassifier(random_state=127, subsample=1)

In [88]:
test_proba = clf.predict_proba(X_actual_test_data)
y_pred = clf.predict(X_actual_test_data)

print(cm:=classification_report(Y_actual_test_data, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_actual_test_data, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_actual_test_data, test_proba[:,1]))

              precision    recall  f1-score   support

           0   0.969625  0.804360  0.879294      7064
           1   0.216553  0.682143  0.328744       560

    accuracy                       0.795383      7624
   macro avg   0.593089  0.743251  0.604019      7624
weighted avg   0.914310  0.795383  0.838855      7624

TN: 5682
FP: 1382
FN: 178
TP: 382
Recall: 0.6821428571428572
Precision: 0.2165532879818594
AUC: 0.787429471363857


In [89]:
clf = RandomForestClassifier(
                         n_estimators=100, 
                         criterion='gini', 
                         max_depth=3, 
                         n_jobs=-1, 
                         max_samples=0.3, 
                         random_state=127
                     )

clf.fit(X_train2, Y_train2)

test_proba = clf.predict_proba(X_actual_test_data)
y_pred = clf.predict(X_actual_test_data)

print(cm:=classification_report(Y_actual_test_data, y_pred, digits=6, target_names=None, output_dict=False))

cm_v = confusion_matrix(Y_actual_test_data, y_pred)
tn,fp,fn,tp = cm_v.ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print("Recall:", tp/(tp+fn))
print("Precision:", tp/(tp+fp))

print("AUC:", roc_auc_score(Y_actual_test_data, test_proba[:,1]))

              precision    recall  f1-score   support

           0   0.951887  0.960646  0.956246      7064
           1   0.438384  0.387500  0.411374       560

    accuracy                       0.918547      7624
   macro avg   0.695135  0.674073  0.683810      7624
weighted avg   0.914169  0.918547  0.916224      7624

TN: 6786
FP: 278
FN: 343
TP: 217
Recall: 0.3875
Precision: 0.4383838383838384
AUC: 0.8653034500889825
